In [8]:
# Text entry with 6 keys: A,S,K,L, space and backspace

# type only 

import numpy as np
import statistics
import math
import time
import stringdist
from operator import itemgetter
import tkinter as tk
from itertools import combinations
import ipywidgets as widgets

In [9]:
widgets.HTMLMath(
    value=r"Compute entropy with probability: $$H(X) = -\sum_{i=1}^{n} P_{i} \log P_{i}$$"
)

In [10]:

# To compute entropy with probability

def H(List):
    sum = 0
    for i in range(len(List)):
        if List[i] != 0: 
            sum = sum + List[i] * math.log(List[i],2)
    return -sum

In [11]:
widgets.HTMLMath(
    value=r"In Info_Measure, we compute equivocation: $$H(X_{1}, ...,X_{m}|Y_{1},...Y_{m}) = \sum_{i=1}^{m} H(X_{i}|X_{i-1}, Y_{i}, Y_{i-1})=H(X_{1}|Y_{1}) + H(X_{2}|X_{1}, Y_{1}, Y_{2}) +...+ H(X_{m}|X_{m-1}, Y_{m-1}, Y_{m})$$ and mutual information: $$I(X_{1}, X_{2},...,X_{m}|Y_{1}, Y_{2},...,Y_{m})= H(X_{1}, X_{2},...,X_{m}) - H(X_{1}, X_{2},...,X_{m}|Y_{1}, Y_{2},...,Y_{m})$$"
)

In [15]:
# Get the probability of a single character and conditional probability conditioning on one previous character

def Info_Measure(List, sen, Y1, t):
    char_unique = set(List)
    single_pro = []
    comb_pro = []

    for cha in char_unique:
        single_pro.append([cha,list(List).count(cha)/len(list(List))])

    #print(single_pro)
    entro_h1 = H(list(map(itemgetter(1), single_pro)))

    unique = []

    for go in range(len(list(List))-1):
        current = [list(List)[go], list(List)[go+1]]
        if current not in unique:
            unique.append(current)
    
    con_pro = []
    
    for everything in unique:
        count = 0
        for fly in range(len(list(List))-1):
            new = [list(List)[fly], list(List)[fly+1]]
            sec = list(List)[fly+1]
            if new == everything:
                count = count + 1
        pro_up = count / (len(list(List))-1)
        for ss in single_pro:
            if ss[0] == sec:
                pro_down = ss[1]
        con_pro.append([everything, pro_up/pro_down])
    
    entro_cond = H(list(map(itemgetter(1), con_pro)))
    
    H_sen = entro_h1 + (len(sen)-1)*entro_cond
    
    ## to compute mutual information between X and Y

    se = []
    un_se = []

    for eachy in Y1:
        sth = list(eachy)
        if sth != 'x':
            un_se.append(sth)
        for q in sth:
            if q != 'x':
                se.append(q)

    #find unique number in se
    unise = set(se)
    #print(unise)

    #find probability of each element in Y
    pro_y_ele = []

    for chay in unise:
            pro_y_ele.append([chay,list(se).count(chay)/len(list(se))])


    match_list = []
    unique_match_list = []

    for s in range(len(list(list(sen)))):
        for i in range(len(un_se)):
            if un_se[i][s] != 'x':
                match_list.append([list(sen)[s], un_se[i][s]])

    for x in match_list:
        if x not in unique_match_list:
            unique_match_list.append(x)

    con_hxy = []
    con_hxy_complete = []

    for each in unique_match_list:
        count = 0
        second = each[1]
        for a in match_list:
            if a == each:
                count = count + 1
        up = count / len(match_list)
        for e in pro_y_ele:
            if e[0] == second:
                down = e[1]
        con_hxy.append(up/down)
        con_hxy_complete.append([each, up/down])

    entro_con_xy1 = 0

    for aa in con_hxy:
        if aa != 0:
            entro_con_xy1 = entro_con_xy1 - aa * math.log(aa,2)

    #print(entro_con_xy1)

    # here compute the other half of H(X|Y)
    to_match_con = []
    to_match_down = []

    for s in range(len(list(sen))-1):
        for i in range(len(un_se)):
            if un_se[i][s] != 'x' and un_se[i][s+1] != 'x':
                to_match_con.append([list(sen)[s], list(sen)[s+1], un_se[i][s], un_se[i][s+1]])
                to_match_down.append([list(sen)[s], un_se[i][s], un_se[i][s+1]])

    # find unique quardplets 
    unique_to_match_con = []
    unique_down = []

    for x in to_match_con:
        if x not in unique_to_match_con:
            unique_to_match_con.append(x)

    for q in to_match_down:
        if q not in unique_down:
            unique_down.append(q)

    #print(unique_down)

    # find each probability
    # for up
    up_p = []
    for unup in unique_to_match_con:
        count = 0
        for q in to_match_con:
            if q == unup:
                count = count + 1
        up_p.append([unup, count/len(to_match_con)])

    #print(up_p)

    down_p = []
    for d in unique_down:
        c = 0
        for s in to_match_down:
            if s == d:
                c = c + 1
        down_p.append([d, c/len(to_match_down)])

    #print(down_p)

    conp = []
    to_test = []

    for fi in range(len(up_p)):
        for ss in range(len(down_p)):
            if up_p[fi][0][0] == down_p[ss][0][0] and up_p[fi][0][2] == down_p[ss][0][1] and up_p[fi][0][3] == down_p[ss][0][2]:
                conp.append(up_p[fi][1]/down_p[ss][1])
                to_test.append(up_p[fi])

    errors = []

    for q in to_test:
        if q[0][0] != q[0][2] or q[0][1] != q[0][3]:
            errors.append(q)

    print("errors come from: ", errors)

    entro_con_xy2 = 0

    for aa in conp:
        entro_con_xy2 = entro_con_xy2 - aa * math.log(aa,2)

    MI = H_sen - entro_con_xy1 - (len(list(sen))-1) * entro_con_xy2
    
    print("Input entropy: ", H_sen, "bits")
    print("Mutual information: ", MI, "bits")
    print("Equivocation: ", entro_con_xy1 - (len(list(sen))-1) * entro_con_xy2, "bits")
    
    if t == 0:
        print("Throught is not available")
    else:
        print("Thoughput: ", MI/t, "bit/s")


In [16]:
## to compute the conventional text entry metrics
def Usual_Measure(stimulus, ainput, time):
    
    # compute word per minute
    
    WPM = len(ainput) / 5 / time * 60
    
    # compute error rate based on levenshtein distance
    er = []
    for einput in ainput:
        if len(einput) > len(stimulus):
            er.append(stringdist.rdlevenshtein(einput, stimulus)/len(einput))
        else:
            er.append(stringdist.rdlevenshtein(einput, stimulus)/len(stimulus))
            
    error_rate = statistics.mean(er)
    
    # compute keystrokes per character
    

    print("Error rate: ", error_rate)
    print("Word Per Minute: ", WPM, "word per minute")
    

In [18]:
L1 = "ask ls ls ask ksa lsk sks las lala sks skl alsk ska"

stimulus1 = "ask las ks"

Y1 = ["ask las ks", "ask las ks"]

Info_Measure(L1,stimulus1,Y1,0)

## TO DO: discuss what the differences between the entropy and perplexity

## TO DO: change the language model to discover the differences

errors come from:  []
Input entropy:  67.7911932778782 bits
Mutual information:  67.7911932778782 bits
Equivocation:  0.0 bits
Throught is not available


In [19]:
start = time.time()
user_type = input("please type: ask las ks ")
end = time.time()
time_spent = end - start

Y1.append(user_type)
Info_Measure(L1,stimulus1,Y1,time_spent)
Usual_Measure(stimulus1, Y1, time_spent)

## TO DO: think about how to deal with other types of errors: omission, transposition, etc...


please type: ask las ks ask las ks
errors come from:  []
Input entropy:  67.7911932778782 bits
Mutual information:  67.7911932778782 bits
Equivocation:  0.0 bits
Thoughput:  26.76632966270586 bit/s
Error rate:  0.0
Word Per Minute:  14.214056741966976 word per minute


## TODO:

* tell each other your text entry rate in bit per second
* implement the intelligent text entry method that you learned earlier
* try a good decoder and a bad decoder to see how the measures change

